# Previsão de Estoque para 28 dias

In [ ]:
import pandas as pd
import numpy as np
from pmdarima import auto_arima
from statsmodels.tsa.stattools import adfuller
from google.cloud import bigquery
from google.oauth2 import service_account
import logging
import matplotlib.pyplot as plt
from datetime import timedelta
from google.cloud import exceptions
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Configurar logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Caminho para o arquivo de chave da conta de serviço
service_account_path = 'tfm-sa.json'
credentials = service_account.Credentials.from_service_account_file(service_account_path)
project_id = 'perseverance-332400'
dataset_id = 'TFM'

# Definir IDs de tabela separadas
data_table_id = 'ds_market'               # Tabela com dados originais
forecast_table_id = 'ds_market_forecast'  # Nova tabela para previsões

full_data_table_id = f'{project_id}.{dataset_id}.{data_table_id}'

# Função para configurar o cliente BigQuery usando credenciais específicas
def initialize_bigquery_client():
    return bigquery.Client(project=project_id, credentials=credentials)

# Função para extrair dados históricos do BigQuery
def get_historical_data_from_bigquery(query: str, client) -> pd.DataFrame:
    try:
        query_job = client.query(query)
        data = query_job.to_dataframe()
        logging.info(f"Dados extraídos do BigQuery com {data.shape[0]} linhas e {data.shape[1]} colunas.")
        logging.info(f"Colunas disponíveis nos dados extraídos: {data.columns.tolist()}")
        return data
    except exceptions.GoogleCloudError as e:
        logging.error(f"Erro ao extrair dados do BigQuery: {e}")
        return pd.DataFrame()  # Retorna um DataFrame vazio para evitar falhas subsequentes

# Função para preparar os dados
def prepare_sales_data(raw_data: pd.DataFrame) -> pd.DataFrame:
    raw_data = raw_data.copy()
    raw_data['date'] = pd.to_datetime(raw_data['date'])
    raw_data['sales'] = raw_data['sales'].fillna(0)
    sales_data = raw_data.groupby('date').agg({'sales': 'sum'}).sort_index()
    if sales_data.isnull().any().any():
        logging.warning("Existem datas sem dados de vendas.")
    sales_data = sales_data.asfreq('D')
    sales_data['sales'] = sales_data['sales'].fillna(0)
    logging.info("Dados de vendas preparados e limpos.")
    return sales_data

# Função para treinar o modelo ARIMA usando pmdarima
def train_forecast_model(data: pd.DataFrame):
    logging.info("Treinando o modelo ARIMA com pmdarima...")
    series = data['sales']

    try:
        model = auto_arima(
            series,
            seasonal=False,
            stepwise=True,
            suppress_warnings=True,
            error_action='ignore',
            trace=True
        )
        order = model.order
        logging.info(f"Modelo treinado com sucesso com ordem {order}.")
        return model, order
    except Exception as e:
        logging.error(f"Erro ao ajustar o modelo ARIMA com pmdarima: {e}")
        raise

# Função para gerar previsões
def predict_sales(data: pd.DataFrame, model, steps=30, forecast_index=None) -> pd.DataFrame:
    logging.info(f"Gerando previsões para {steps} dias...")
    if model is None:
        raise ValueError("Modelo inválido. Não é possível gerar previsões.")
    forecast = model.predict(n_periods=steps)
    if forecast_index is not None:
        forecast_df = pd.DataFrame({'date': forecast_index, 'forecast_sales': forecast})
    else:
        forecast_df = pd.DataFrame({
            'date': pd.date_range(data.index[-1] + timedelta(days=1), periods=steps),
            'forecast_sales': forecast
        })
    logging.info("Previsão gerada com sucesso.")
    return forecast_df

# Função para carregar previsões no BigQuery
def store_forecast_results(data: pd.DataFrame, table_id: str, client):
    logging.info(f"Carregando previsões para a tabela {table_id} no BigQuery...")
    
    # Remover coluna 'actual_sales' se ela não for necessária na tabela de previsões
    if 'actual_sales' in data.columns:
        data = data.drop(columns=['actual_sales'])
    
    job_config = bigquery.LoadJobConfig(write_disposition="WRITE_APPEND")  # Evita sobrescrever a tabela original
    job = client.load_table_from_dataframe(data, table_id, job_config=job_config)
    job.result()
    logging.info("Previsões carregadas com sucesso no BigQuery.")


# Função para plotar previsões vs. dados reais
def plot_forecast_with_actual(train_data: pd.DataFrame, test_data: pd.DataFrame, forecast: pd.DataFrame, store: str, item: str):
    plt.figure(figsize=(12, 6))
    plt.plot(train_data.index, train_data['sales'], label='Dados de Treinamento')
    plt.plot(test_data.index, test_data['sales'], label='Vendas Reais (Teste)')
    plt.plot(forecast['date'], forecast['forecast_sales'], label='Previsão', linestyle='--')
    plt.legend()
    plt.title(f'Previsão vs. Vendas Reais - Loja {store} Item {item}')
    plt.xlabel('Data')
    plt.ylabel('Vendas')
    plt.show()

# Pipeline completo para previsão de estoque dos itens por loja
def run_forecast_pipeline_for_top_items_per_store(query: str, forecast_table_full_id: str):
    client = initialize_bigquery_client()
    raw_data = get_historical_data_from_bigquery(query, client)

    # Verificar se as colunas necessárias existem
    required_columns = {'store', 'item', 'date', 'sales'}
    if not required_columns.issubset(raw_data.columns):
        missing = required_columns - set(raw_data.columns)
        raise ValueError(f"As colunas a seguir estão faltando nos dados: {missing}")

    # Obter lista de lojas únicas
    stores = raw_data['store'].unique()
    all_forecasts = []

    for store in stores:
        logging.info(f"\nProcessando loja {store}...")
        store_data = raw_data[raw_data['store'] == store]
        total_sales_per_item = store_data.groupby('item')['sales'].sum().reset_index()
        top_items = total_sales_per_item.sort_values(by='sales', ascending=False).head(1)['item']  # número de itens a serem previstos

        for item in top_items:
            logging.info(f"Processando item {item} na loja {store}...")
            item_data = store_data[store_data['item'] == item]
            prepared_data = prepare_sales_data(item_data)

            if len(prepared_data) < 2:
                logging.warning(f"Dados insuficientes para item {item} na loja {store}, pulando...")
                continue

            try:
                # Dividir os dados em treinamento e teste
                train_data = prepared_data[prepared_data.index < '2015-12-01']
                test_data = prepared_data[prepared_data.index >= '2015-12-01']

                if len(train_data) < 2 or len(test_data) == 0:
                    logging.warning(f"Dados insuficientes para treinamento ou teste para item {item} na loja {store}, pulando...")
                    continue

                model, order = train_forecast_model(train_data)
                if model is None:
                    logging.error(f"Não foi possível ajustar o modelo para item {item} na loja {store}.")
                    continue

                # Prever para o período de teste
                steps = len(test_data)
                forecast = predict_sales(train_data, model, steps=steps, forecast_index=test_data.index)
                forecast['store'] = store
                forecast['item'] = item

                # Comparar previsões com vendas reais
                forecast['actual_sales'] = test_data['sales'].values

                # Calcular métricas de avaliação
                rmse = mean_squared_error(forecast['actual_sales'], forecast['forecast_sales'], squared=False)
                mae = mean_absolute_error(forecast['actual_sales'], forecast['forecast_sales'])
                mape = np.mean(np.abs((forecast['actual_sales'] - forecast['forecast_sales']) / (forecast['actual_sales'] + 1e-5))) * 100  # Evitar divisão por zero

                logging.info(f"Loja {store}, Item {item} - RMSE: {rmse:.2f}, MAE: {mae:.2f}, MAPE: {mape:.2f}%")

                # Adicionar informações ao DataFrame de previsões
                forecast['rmse'] = rmse
                forecast['mae'] = mae
                forecast['mape'] = mape
                forecast['order'] = str(order)

                all_forecasts.append(forecast)

                # Plotar previsões vs. vendas reais
                plot_forecast_with_actual(train_data, test_data, forecast, store, item)

            except Exception as e:
                logging.error(f"Erro ao processar item {item} na loja {store}: {e}")
                continue

    if all_forecasts:
        final_forecast = pd.concat(all_forecasts, ignore_index=True)
        # Armazenar previsões na tabela de previsões
        store_forecast_results(final_forecast, forecast_table_full_id, client)
    else:
        logging.warning("Nenhuma previsão foi gerada.")

# Definir a consulta SQL para carregar os dados de 2015
query = f"""
SELECT *
FROM `{full_data_table_id}`
WHERE EXTRACT(YEAR FROM date) = 2015
"""

# Executar o pipeline, especificando a tabela de previsões
run_forecast_pipeline_for_top_items_per_store(
    query,
    f'{project_id}.{dataset_id}.{forecast_table_id}'
)

In [ ]:
import pandas as pd
import numpy as np
import itertools
from pmdarima.arima import ARIMA
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
import logging

# Configuração de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Função para verificar estacionariedade e aplicar diferenciação, se necessário
def check_and_differentiate(data: pd.DataFrame, column: str) -> pd.DataFrame:
    result = adfuller(data[column])
    logging.info(f"ADF Statistic: {result[0]}, Valor-p: {result[1]}")
    if result[1] > 0.05:
        logging.info("Série não estacionária. Aplicando diferenciação.")
        data[column] = data[column].diff().dropna()
    else:
        logging.info("Série já é estacionária.")
    return data

# Função para normalizar os dados de vendas
def normalize_sales_data(data: pd.DataFrame, column: str) -> tuple:
    scaler = MinMaxScaler()
    data[column] = scaler.fit_transform(data[[column]])
    logging.info("Dados de vendas normalizados com MinMaxScaler.")
    return data, scaler

# Função para otimizar o modelo ARIMA testando diferentes combinações de parâmetros
def optimize_arima(data: pd.Series):
    p = d = q = range(0, 3)
    pdq = list(itertools.product(p, d, q))

    best_aic = float("inf")
    best_order = None
    best_model = None

    for order in pdq:
        try:
            model = ARIMA(order=order)
            model.fit(data)
            aic = model.aic()
            if aic < best_aic:
                best_aic = aic
                best_order = order
                best_model = model
        except:
            continue

    logging.info(f"Melhor modelo ARIMA: ordem {best_order} com AIC {best_aic}")
    return best_model

# Função para validação cruzada em séries temporais
def cross_validate_arima(data: pd.Series, model_func, splits=5):
    tscv = TimeSeriesSplit(n_splits=splits)
    errors = []

    for train_index, test_index in tscv.split(data):
        train, test = data.iloc[train_index], data.iloc[test_index]
        model = model_func(train)
        forecast = model.predict(n_periods=len(test))
        rmse = mean_squared_error(test, forecast, squared=False)
        errors.append(rmse)

    avg_rmse = np.mean(errors)
    logging.info(f"RMSE médio com validação cruzada: {avg_rmse}")
    return avg_rmse

# Função para calcular as métricas de erro
def calculate_metrics(actual, forecast):
    rmse = mean_squared_error(actual, forecast, squared=False)
    mae = mean_absolute_error(actual, forecast)
    mape = np.mean(np.abs((actual - forecast) / (actual + 1e-5))) * 100  # evitar divisão por zero

    logging.info(f"RMSE: {rmse}, MAE: {mae}, MAPE: {mape}%")
    return rmse, mae, mape

# Função principal para executar o pipeline de previsão
def run_forecast_pipeline(data: pd.DataFrame, column: str):
    # Verificar e diferenciar para estacionariedade
    data = check_and_differentiate(data, column)

    # Normalizar os dados
    data, scaler = normalize_sales_data(data, column)

    # Otimizar ARIMA e validar o modelo
    model = optimize_arima(data[column])
    rmse_cv = cross_validate_arima(data[column], lambda x: optimize_arima(x))

    # Gerar previsão para os próximos 28 dias
    forecast = model.predict(n_periods=28)
    forecast = scaler.inverse_transform(forecast.values.reshape(-1, 1)).flatten()  # desfazendo a normalização

    # Calcular métricas de erro usando os últimos 28 dias de dados reais
    actual = scaler.inverse_transform(data[column].values[-28:].reshape(-1, 1)).flatten()
    rmse, mae, mape = calculate_metrics(actual, forecast)

    return forecast, {"rmse": rmse, "mae": mae, "mape": mape, "rmse_cv": rmse_cv}

# Exemplo de execução do pipeline
if __name__ == "__main__":
    # Suponha que você tenha um DataFrame `df` com uma coluna `sales` para rodar o pipeline
    df = pd.DataFrame({
        'date': pd.date_range(start='1/1/2015', periods=1000),
        'sales': np.random.randint(0, 100, size=1000)
    }).set_index('date')

    forecast, metrics = run_forecast_pipeline(df, 'sales')
    print("Previsão para os próximos 28 dias:", forecast)
    print("Métricas de erro:", metrics)
